# Homework 2

In [38]:
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.mixture import GaussianMixture
from IPython.display import Image  
from scipy.stats import norm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydotplus
import seaborn as sns

## Load Data
The feature space is already lagged. Therefore, you do not need to lag the variables yourself. The data track these companies over 3 years (2018-2020). We will train the data in 2018, validate in 2019, and forecast 2020. Further instructions are given in the questions.


In [2]:
dat = pd.read_csv('hw2_data.csv')

In [3]:
dat

,permno,fyear,label,ret,acc,agr,cfp,ep,ni,op,...,grltnoa,conv,operprof,capxint,chpm,alm,hire,herf,bm_ia,me_ia
0,15417,2018.0,0,0.102609,0.053313,0.252472,0.253973,0.148027,0.095118,0.323725,...,0.123393,0.0,0.467402,0.089973,-0.016902,0.469687,-0.050260,0.553758,1.622149,0.017834
1,89031,2018.0,1,0.067922,-0.001165,0.215518,0.005945,0.002326,0.012251,0.126233,...,0.024414,0.0,0.140084,0.033813,-0.049337,0.084011,0.215316,0.759126,0.318117,0.127617
2,11154,2018.0,0,-0.071429,0.000948,0.032913,0.184267,0.024266,0.008571,0.155644,...,0.114552,1.0,0.163587,0.091550,-0.066428,0.144939,-0.035948,0.482831,2.015340,0.072533
3,13556,2018.0,0,-0.326014,0.045752,-0.095131,-1.258587,-1.340418,1.527239,-1.582117,...,0.117086,0.0,-2.972048,0.020384,1.954118,-0.075192,-0.234043,0.114982,0.965078,0.001005
4,14296,2018.0,0,0.249443,0.016353,-0.020653,0.519260,0.449600,0.854538,-0.183793,...,0.329045,0.0,0.982905,0.014257,0.821456,0.269721,-0.048576,0.223411,1.151220,0.039338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238,14532,2020.0,-1,-0.080498,-0.064369,-0.050658,0.064593,0.028561,0.000000,-0.213658,...,0.000831,0.0,-0.226537,0.000677,0.000912,0.683570,-0.010638,0.383695,1.508840,0.011199
2239,41355,2020.0,0,0.015946,0.043579,0.147297,0.082415,0.064142,-0.030160,0.383195,...,0.002844,0.0,0.402497,0.011861,0.031440,0.238843,-0.027287,0.964868,0.199428,2.990846
2240,13983,2020.0,0,0.112274,0.003504,-0.219388,-0.345708,-0.429181,0.004651,0.196534,...,-0.196021,0.0,0.116354,0.021174,-0.419594,0.178488,-0.050000,0.505660,1.127711,0.139232
2241,47619,2020.0,0,-0.057480,0.007940,-0.033335,-0.010621,-0.123947,0.000000,0.096157,...,-0.016991,0.0,0.093207,0.034593,-0.014232,0.726728,-0.134865,0.242645,3.577947,0.009458


## Generative Learning

#### 1. Keep only the labels between -1 and 3. 
Split the data into Train-Validation-Test:
- Training data should contain features in 2018, do not forget to remove ‘label’
- Training labels should only contain ‘label’ in 2018
- Validation data should contain features in 2019, do not forget to remove ‘label’ • Validation labels should only contain ‘label’ in 2019
- Test data should contain features in 2020, do not forget to remove ‘label’
- Test labels should only contain ‘label’ in 2020

In [4]:
# filter label between -1 and 3
dat_1 = dat.loc[(dat['label'] >= -1) & (dat['label'] <= 3)]

# Training data
train_feature = dat_1.loc[dat_1['fyear'] == 2018]
train_label = train_feature['label']
train_feature = train_feature.drop(columns=['label', 'permno', 'fyear'])

# validation data
valid_feature = dat_1.loc[dat_1['fyear'] == 2019]
valid_label = valid_feature['label']
valid_feature = valid_feature.drop(columns=['label', 'permno', 'fyear'])

# test data
test_feature = dat_1.loc[dat_1['fyear'] == 2020]
test_label = test_feature['label']
test_feature = test_feature.drop(columns=['label', 'permno', 'fyear'])

#### 2. Compute and report the prior probabilities $\pi_j$ for all labels in the Training set.

In [5]:
all_label = pd.DataFrame(train_label).groupby('label').size()
all_label = pd.DataFrame(all_label, columns=['count'], index=all_label.index)
all_label['prior'] = all_label['count'] / sum(all_label['count'])

pi = all_label['prior']
pi

label
-1    0.069638
 0    0.814299
 1    0.088208
 2    0.020427
 3    0.007428
Name: prior, dtype: float64

#### 3. Using the Training set, calculate the likelihood for feature ‘ret’ to be 0.1 conditional on each value of the label $P_j = P(ret = 0.1|y = j)$ (Use the Normal PDF found in the following link: [Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html])). Report the density for each value of the label. You need to code this by hand, 0 points will be given if you use the pre-coded scipy function.

In [6]:
train_feature

,ret,acc,agr,cfp,ep,ni,op,rsup,cash,chcsho,...,grltnoa,conv,operprof,capxint,chpm,alm,hire,herf,bm_ia,me_ia
0,0.102609,0.053313,0.252472,0.253973,0.148027,0.095118,0.323725,0.907821,0.170630,0.099789,...,0.123393,0.0,0.467402,0.089973,-0.016902,0.469687,-0.050260,0.553758,1.622149,0.017834
1,0.067922,-0.001165,0.215518,0.005945,0.002326,0.012251,0.126233,0.033752,0.242426,0.012327,...,0.024414,0.0,0.140084,0.033813,-0.049337,0.084011,0.215316,0.759126,0.318117,0.127617
2,-0.071429,0.000948,0.032913,0.184267,0.024266,0.008571,0.155644,-0.060579,0.301746,0.008608,...,0.114552,1.0,0.163587,0.091550,-0.066428,0.144939,-0.035948,0.482831,2.015340,0.072533
3,-0.326014,0.045752,-0.095131,-1.258587,-1.340418,1.527239,-1.582117,0.020306,0.050378,-0.539456,...,0.117086,0.0,-2.972048,0.020384,1.954118,-0.075192,-0.234043,0.114982,0.965078,0.001005
4,0.249443,0.016353,-0.020653,0.519260,0.449600,0.854538,-0.183793,0.033630,0.022905,-0.764971,...,0.329045,0.0,0.982905,0.014257,0.821456,0.269721,-0.048576,0.223411,1.151220,0.039338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,0.054105,-0.001807,0.089017,0.107095,0.041666,0.013848,0.142874,0.040472,0.012117,0.013944,...,0.109585,0.0,0.201967,0.032356,0.036884,0.011584,0.148718,1.000000,1.000000,1.000000
1081,0.079388,0.000527,0.015823,0.047288,0.031791,0.008216,0.201411,0.014678,0.041941,0.008250,...,0.051059,0.0,0.257229,0.028789,0.050255,0.096617,0.099057,0.100118,0.677892,0.613586
1082,-0.148534,-0.010133,-0.236462,-1.358676,-1.638366,0.000000,-0.039809,-0.200483,0.046370,0.000000,...,-0.145097,0.0,-0.029605,0.001979,-0.869378,-0.559189,-0.231169,0.100118,10.746082,0.002548
1083,0.056180,-0.000269,-0.120499,0.080167,-0.018595,0.011872,0.108467,-0.441751,0.069532,0.011943,...,-0.002094,0.0,0.112288,0.002194,0.082869,0.483436,-0.086294,0.091165,2.094959,0.032947


In [7]:
train = dat_1.loc[dat_1['fyear'] == 2018]
ret_mean = train.groupby(['label']).mean()['ret']
ret_var = train.groupby(['label']).var()['ret']

P = pd.DataFrame()
P['mean'] = ret_mean
P['var'] = ret_var
P['likelihood'] = norm.pdf((0.1 - P['mean']) / np.sqrt(P['var']))
P

,mean,var,likelihood
label,,,
-1,0.014262,0.013064,0.301106
0,0.010048,0.010146,0.267751
1,0.027497,0.012392,0.322697
2,-0.010655,0.008859,0.199889
3,0.068194,0.021344,0.389599


#### 4. Use Guassian naive bayes from the scikit-learn library (found here:[scikitlearnfunction](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes)) to classify the test data. Report the accuracy. You need to use the train+validation set.

In [8]:
gnb = GaussianNB()
y_pred = gnb.fit(train_feature, train_label).predict(valid_feature)
accuracy = accuracy_score(valid_label, y_pred)
print(f'accuracy is {round(accuracy, ndigits=4)}')

accuracy is 0.269


#### 5. Compute the confusion matrix (as shown in the lectures) and report the top 2 pairs with most (absolute number) incorrect classifications.

In [9]:
mat = confusion_matrix(valid_label, y_pred, labels=[i for i in range(-1, 4)])
mat

array([[ 15,  22,   2,  20,  16],
       [ 25, 244,  22, 240, 328],
       [  1,  20,  12,  16,  39],
       [  1,   1,   3,   9,   9],
       [  0,   1,   1,   2,   3]])

In [37]:
mat = confusion_matrix(valid_label, y_pred, labels=[i for i in range(-1, 4)])

def find_n_max(matrix, n):

    pairs = []

    for i in range(n):

        max_incorrect = 0
        max_row = 0
        max_col = 0

        for row in range(len(matrix)):
            
            for column in range(len(matrix[0])):
                
                val = matrix[row, column]
                
                if (val > max_incorrect) & (row != column):

                    max_incorrect = val
                    max_row = row
                    max_col = column
                
        matrix[max_row, max_col] = 0 # change the biggest value to 0 and loop again
        pairs.append([max_row, max_col]) # row and column
        
    return pairs

find_n_max(mat, 2)

[[ 15  22   2  20  16]
 [ 25 244  22 240 328]
 [  1  20  12  16  39]
 [  1   1   3   9   9]
 [  0   1   1   2   3]]


[[1, 4], [1, 3]]

Based on the data given by `pairs`, we know that the top 2 pairs with most incorrect are [3, 0] and [2, 0].

#### 6. Implement Gaussian Mixture model on the data as shown in class. 
Tune the covariance type parameter on the validation data. Use the selected value to compute the test accuracy. As always, train the model on train+validation data to compute the test accuracy. Train the model twice, the first model should use the covariance type that yielded the highest accuracy in the validation stage. The second model should use the covariance type that yielded the second highest accuracy in the validation stage. Comment on the accuracy on the test set of the the models you ran. Hint: Use ‘n components=3, init params=”kmeans”, random state=34’.

In [ ]:
clf = GaussianMixture(n_components=3, init_params='kmeans', random_state=34)
clf.fit(train_feature, train_label)


#### 7. Bonus Question: Apply Linear Discriminant Analysis model on the train+validation data and report the accuracy obtained on test data. Report the transformation matrix (w) along with the intercept.